# Run Diffdock NIM

In [3]:
import requests
import os
import numpy as np

# Utils

In [4]:
# utility function to prepare (start clean) the output directory
def prepare_directory(temp):
    import os, shutil
    """
    Create a new directory and delete the old one if it exists
    :param temp: str: path to the directory
    """
    if os.path.exists(temp):
        # Remove the directory and all its contents
        shutil.rmtree(temp)
    # Recreate the directory
    os.makedirs(temp)


# Set up the urls

In [5]:
# if NIM is launched in a remote server, replace localhost with the IP address of your instance
base_url = "http://localhost:8000" 
query_url = base_url + "/molecular-docking/diffdock/generate"
health_check_url = base_url + "/v1/health/ready"

In [6]:
# run health check
response = requests.get(health_check_url)
assert response.text == "true", "Health check failed"

# Usage case 1: single GPU, small dataset

This section is for running NIMs on single GPU in instance. It will also work if you have multiple-GPU instance, but because we are not using `asyncio`, you will be utilizing only GPU:0 instead of all the GPUs. Refer to the the next section for running on multiple GPUs 

## Input file structure

Prepare a file structure like this: 

```bash
batch_input_small/
├── input_smiles.txt
├── receptor.pdb
```

Specifically: 
- `input_smiles.txt`: a txt file with SMILES strings, one per line. 
- `receptor.pdb`: a PDB file of the target

**Example files**: See `batch_input_small` folder. These are taken from [DUD-E](https://dude.docking.org/). The example has 3 compounds for aa2ar

## Run prediction

In [7]:
def submit_query(protein_file_path, ligand_file_path, num_poses=20, time_divisions=20, steps=18):
    """
    Submit a query to the server
    :param protein_file_path: path to the protein file, must be a PDB file
    :param ligand_file_path: path to the ligand file, must be a txt, SDF, MOL2, file. If using batch-docking, only txt and SDF are supported. 
    :param num_poses: int, number of poses to be generated, default is 20
    :param time_divisions: int, number of time divisions, default is 20
    :param steps: int, number of diffusion steps, default is 18
    :return: dict of response, status code and JSON response content if successful, otherwise return status code and error message
    """

    with open(protein_file_path, 'r') as file:
        protein_bytes = file.read()
    with open(ligand_file_path, 'r') as file:
        ligand_bytes = file.read()

    ligand_file_type = ligand_file_path.split('.')[-1]
    
    data = {
        "ligand": ligand_bytes,
        "ligand_file_type": ligand_file_type, # txt, sdf, mol2
        "protein": protein_bytes,
        "num_poses": num_poses,
        "time_divisions": time_divisions,
        "steps": steps,
        "save_trajectory": False, # diffusion trajectory
        "is_staged": False
    }
    
    headers = {"Content-Type": "application/json"}

    with requests.post(query_url, headers=headers, json=data) as response:
        status_code = response.status_code
        try:
            response.raise_for_status() # optional, immediately fails if the status code is not 200
            output = {
                "status_code": status_code,
                "response": response.json()
            }
        except:
            output = {
                "status_code": status_code,
                "response": response.text()   
            }
    
    return output


Now we will run prediction with 3 ligands in the `input_smiles.txt` file: 
1. Each of the ligands will be docked to the protein in `receptor.pdb` file. 
2. The number of poses will be 5. 
3. The number of time divisions will be 20. 
4. The number of diffusion steps will be 18. 

In [8]:
%%time
result = submit_query(
    protein_file_path="data/batch_input_small/receptor.pdb",
    ligand_file_path="data/batch_input_small/input_smiles.txt",
    num_poses=5, 
    time_divisions=20, 
    steps=18
)

CPU times: user 5.6 ms, sys: 499 μs, total: 6.09 ms
Wall time: 7.47 s


## Analyze results

In [9]:
result.keys()

dict_keys(['status_code', 'response'])

we will look at the `response`key

In [10]:
response = result['response']
response.keys()

dict_keys(['trajectory', 'ligand_positions', 'position_confidence', 'status', 'protein', 'ligand'])

### Status

This shows the success or failed status of each ligand

In [11]:
# 3 input ligand, all showed success
response['status']

['success', 'success', 'success']

### Trajectory

`trajectory`is the diffusion trajectory. Because we set `"save_trajectory": False`, it is just empty lists: 

In [12]:
response['trajectory']

[['', '', '', '', ''], ['', '', '', '', ''], ['', '', '', '', '']]

### Ligand poses

`ligand_positions` are the predicted poses. Becuase we set `num_poses=5`, for each ligand we will have 5 poses: 

In [13]:
poses = response['ligand_positions']

assert len(poses) == 3 # we have 3 ligandsin the `input_smiles.txt` file. 

# let's look at the first ligand. It has 5 predicted poses
assert len(poses[0]) == 5

# let's look at the first pose of the first ligand. 
poses[0][0]

'protein_ligand_0\n     RDKit          3D\n\n 31 36  0  0  0  0  0  0  0  0999 V2000\n   -8.8297   -4.8035   55.3940 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -8.7538   -6.1972   55.1800 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -9.3504   -7.0648   56.0156 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -9.2966   -8.3960   55.8398 C   0  0  0  0  0  0  0  0  0  0  0  0\n   -9.9426   -9.2722   56.7478 N   0  0  0  0  0  0  0  0  0  0  0  0\n   -9.8324  -10.7027   56.7085 C   0  0  0  0  0  0  0  0  0  0  0  0\n  -10.0840  -11.2111   58.1147 C   0  0  0  0  0  0  0  0  0  0  0  0\n  -11.4144  -10.8973   58.5821 N   0  0  0  0  0  0  0  0  0  0  0  0\n  -11.4776  -11.1118   59.9876 C   0  0  0  0  0  0  0  0  0  0  0  0\n  -12.9158  -10.9790   60.4411 C   0  0  1  0  0  0  0  0  0  0  0  0\n  -12.9819  -11.2681   61.9465 C   0  0  0  0  0  0  0  0  0  0  0  0\n  -12.3175  -12.5142   62.2553 N   0  0  0  0  0  0  0  0  0  0  0  0\n  -11.6049  -12.8801   63.3343 C   0  0  0  0  0  0  0  0  0  

### Pose confidence score

`position_confidence` is the confidence score of each pose. The higher the better. A confidence score of 0 means that there is 50% chance the predicted pose is within RMSD< 2A from the ground truth. 

By default, the `ligand_positions` is a 1:1 match with the `position_confidence`. For example, the 1st pose in the `ligand_positions` matches to the 1st score in the `position_confidence`, the 2nd pose matches to the 2nd score, and so on. 

By default, the `position_confidence` is sorted from high (best) to low (worst). This means that, the 1st pose in `ligand_positions` is the one with the highest confidence score. 


In [14]:
# this is a list of 3 input ligands
assert len(response['position_confidence']) == 3

# each sublist has 5 scores, one score for each pose
assert len(response['position_confidence'][0]) == 5

# here is what the confidence scores of the 1st ligand looks like
response['position_confidence'][0]

[0.12911391258239746,
 0.07320181280374527,
 -0.00487934798002243,
 -0.07097923755645752,
 -3.1280288696289062]

The actual confidence scores will be different in each run

### Writing the output file

We need to write the poses to SDF file. One way to do this is to combine all poses of all ligands into one single SDF file. This is especially helpful if you have a large number of ligands the input file. Combining the predicted poses into one file can save some file operations, e.g. if you want to copy the data to another location. 

To do this, we will: 
1. filter the ligands by the status. We don't want to write the failed ligands to file. 
2. For all the successful ligands, we will write the poses to a single SDF file. 
3. The poses will be named as `ligand_0_pose_0`, `ligand_0_pose_1` etc
4. Diffdock poses are not guaranteed to be valid. So we will use RDKit to sanitize the molecules before writing them to file. 
5. We will save the confidence score as a property `Confidence` of the molecule. 

Below is a script that does this. 

First, install RDKit: 
```bash
pip install rdkit
```

In [15]:
from rdkit import Chem
from rdkit.Chem import AllChem, SDWriter

# change it to your desired output directory. use the `prepare_directory` function to start clean. 
output_dir = "output/batch_output_small" 
prepare_directory(output_dir) 

# output SDF file which has all ligands and all poses
output_sdf_file = os.path.join(output_dir, 'output.sdf') # output file path

# create a writer
writer = SDWriter(output_sdf_file)

# select indices where status is succes
status_array = np.array(response['status'])
success_indices = np.where(status_array == 'success')[0]

for i in success_indices:
    # e.g. ligand ID will be ligand_0, ligand_1, etc
    ligand_id = 'ligand_' + str(i)

    # get the ligand poses and confidence scores
    ligand_positions = response['ligand_positions'][i]
    confidence_scores = response['position_confidence'][i]

    # write to SDF file
    for idx, sdf_str in enumerate(ligand_positions):
        mol = Chem.MolFromMolBlock(sdf_str)
        if mol:
            try: 
                # sanitize the molecule
                Chem.SanitizeMol(mol)
                # set the name, like ligand_0_pose_0, ligand_0_pose_1, etc 
                mol.SetProp("_Name", ligand_id+f'_pose_{str(idx)}')
                # set the confidence score as a property
                mol.SetProp("Confidence", str(np.round(confidence_scores[idx], 4)))
                writer.write(mol)
            except:
                print(f"Failed to sanitize molecule {ligand_id}_pose_{str(idx)}")
                continue


To visualize the results in Pymol, load the `output.sdf` file and the `receptor.pdb` file. You will be able to cycle through the poses like this: 

![](../images/Diffdock_result.png)

### Input protein and ligand files

In [23]:
# string, input protein PDB file
response['protein']

'ATOM      1  N   ILE     3     -30.582 -20.763  57.829\nATOM      2  CA  ILE     3     -29.314 -20.499  57.159\nATOM      3  C   ILE     3     -28.389 -19.651  58.027\nATOM      4  O   ILE     3     -28.839 -18.751  58.736\nATOM      5  CB  ILE     3     -29.525 -19.801  55.801\nATOM      6  CG1 ILE     3     -30.369 -18.537  55.975\nATOM      7  CG2 ILE     3     -30.184 -20.750  54.811\nATOM      8  CD1 ILE     3     -30.644 -17.800  54.681\nATOM      9  N   MET     4     -27.095 -19.947  57.964\nATOM     10  CA  MET     4     -26.097 -19.237  58.759\nATOM     11  C   MET     4     -25.637 -17.954  58.075\nATOM     12  O   MET     4     -25.728 -17.821  56.855\nATOM     13  CB  MET     4     -24.892 -20.141  59.029\nATOM     14  CG  MET     4     -25.165 -21.271  60.008\nATOM     15  SD  MET     4     -25.242 -20.704  61.718\nATOM     16  CE  MET     4     -23.568 -20.112  61.954\nATOM     17  N   GLY     5     -25.139 -17.013  58.871\nATOM     18  CA  GLY     5     -24.637 -15.757 

In [24]:
# string, input ligand file
response['ligand']

'Nc5nc(N3CCN2C[C@@H](Cn1ccnc1)CC[C@H]2C3)nc6nc(c4ccco4)nn56\nCN4CCC(CC(=O)Nc3cc(n1nc(C)cc1C)nc(c2ccc(C)o2)n3)CC4\nCOc5ccc(NC(=O)Nc3nc1nn(C)c(SC)c1c4nc(c2ccco2)nn34)cc5'

# Usage case 2: multi GPU, large dataset

In a multi-GPU instance, triton will handle concurrent requests and distribute them across the GPUs. 

Here is one way to do this: 

1. Split our input file into `N` batches, where `N` = number of GPUs on your instance.
2. Use `asyncio` to simulataneoulsy sumbit these batches to the server. 
3. Triton server will parallelize the requests to the GPUs and return the results of each GPU. 
4. We will process the results of each GPU and combine them into a single SDF file. 

## Input file structure

Prepare a file structure like this: 

```bash
batch_input_large/
├── target1
    |--receptor.pdb
    |--batches
        |--batch_0
            |--status.csv
        |--batch_1
            |--status.csv
        |--batch_2
            |--status.csv
        |--batch_3
            |--status.csv
        |--batch_4
            |--status.csv
        |--batch_5
            |--status.csv
        |--batch_6
            |--status.csv
        |--batch_7
            |--status.csv
|--target2
...
```

Specifically:  
- `receptor.pdb`: a PDB file of the target
- `batches`: a folder that contains multiple batches of SMILES files. Each batch is a folder that contains a `status.csv` file. The number of batches is the same as number of GPUs. 
- `status.csv`: a CSV file that contains the status of each compound in the batch. See the next section for more details. 

**Example files**: See `batch_input_large` folder. 

## Prepare status.csv

When running many ligands at once, it could be helpful to track the status (success, failed, timeout etc) of each ligand. In this case, instead of directly using txt file as input, we will first build a CSV file like this: 

```csv
molecule_id,category,smiles,status,fail
0,active,Nc5nc(N3CCN2C[C@@H](Cn1ccnc1)CC[C@H]2C3)nc6nc(c4ccco4)nn56,pending,0
1,active,CN4CCC(CC(=O)Nc3cc(n1nc(C)cc1C)nc(c2ccc(C)o2)n3)CC4,pending,0
2,active,COc5ccc(NC(=O)Nc3nc1nn(C)c(SC)c1c4nc(c2ccco2)nn34)cc5,pending,0
```

- `molecule_id`: unique ID of the molecule. 
- `category`: optional, can be a field used as some kind of notes. In this case I labeled the compound as `active` or `decoy`
- `smiles`: SMILES
- `status`: status of the molecule: `pending`: not predicted. `success`: predicted. `timeout`: timedout
- `fail`: used as a flag to indicate if the molecule failed to predict. 0: not failed. 1: failed. 


**Tips**: 
1. I recommend splitting each batch **roughly** the same size. For example, if target1 has 80 compounds to predict, and we have 8 GPUs, then we will split its compound into 8 batches of 10 compounds/batch. Then we look at target2, if it has 800 compounds to predict, then we will split into 8 batches of 100 compounds/batch. Etc. 
2. In general, as you increase the batch size (e.g. number of ligands in each batch), the per-ligand prediction time will decrease. So it is desirable to use as a large batch size as possible. In practice, there are some considerations :
    - By default, there is a limit of 4 hours per batch. This number can be changed by going into `/opt/inference.py` INSIDE the container, and modify the `TRITON_TIMEOUT` variable. So if you batch size is too large, per-batch prediction might go beyond this time limit, then you will get timeout errors.  
    - Some GPUs might have smaller memory. If the batch size is too large, you might get out of memory erros 
    - Because of these factors, it is recommended that you try out different batch size to make sure the prediction doesn't time out and the GPU is fully utilized.
    - Because we are iterating by target, a smaller batch might finish earlier, then its GPU becomes idle and will need to wait for the larger batch to finish.  Therefore, keeping the batches of the same target roughly the same size helps to uitilize the GPU more efficiently. 
 


**Example files**: See `batch_input_large` folder. These are taken from [DUD-E](https://dude.docking.org/). The example has 80 compounds for aa2ar, and 40 compounds for abl1.

## Run prediction

First install additional packages: 

```bash
pip install aiofiles aiohttp loguru rdkit
```

Then run the prediction with 
```bash
python3 run_diffdock_asyncio.py
```

## Analyze the results

### Output files

The output file is structured like this: 

```bash
output
|--batch_output_large
    |--5_poses_trial_0
        |--aa2ar
            |--batch_0
                |--output.sdf
                |--status.csv
            |--batch_1
                |--output.sdf
                |--status.csv
            |--batch_2
                |--output.sdf
                |--status.csv
            |--batch_3
                |--output.sdf
                |--status.csv
            |--batch_4
                |--output.sdf
                |--status.csv
            |--batch_5
                |--output.sdf
                |--status.csv
            |--batch_6
                |--output.sdf
                |--status.csv
            |--batch_7
                |--output.sdf
                |--status.csv
            |--log.log
            |--time.pkl
        |--ab1
        ...
```

The actual folder names might differ depending on how you set it up in the python script. 

Specifically: 
- `output.sdf`: this is the file containing the poses of each ligand in the batch. The ID of the molecule is similar to this format: `active_ligand_id_pose_id`. The confidence score is stored as the `Confidence` property. 
- `status.csv`: this is the updated `status.csv` file containing the status of each ligand in the batch.
- `log.log`: loguru logs. 
- `time.pkl`: This pickle file contains the total runtime in seconds (`total_runtime_sec`) and failed batch ids (`failed_batches`). The total runtime is the wall time of all batches in that specific target. The failed batches is the ID of the batches that the server returned a non-200 status code. 


### Runtime

Below is the runtime using Diffodck version `nvcr.io/nim/mit/diffdock:1.2.0`


| instance | number of poses | dataset | total run time (sec) | per ligand runtime (sec) |
| -------- | ---------- | -------- | -------- | -------- |
| AWS g5.48xlarge (8xA10) | 5 | aa2ar example | 19.3 | 0.24|
| AWS g5.48xlarge (8xA10) | 5 | abl1 example | 10.9 |  0.27 |
| AWS p3.16xlarge (8xV100)	 | 5 | aa2ar example |24.1 | 0.30 |
| AWS p3.16xlarge (8xV100) | 5 | abl1 example| 14.3 | 0.36 |
| DGXC 8xA100 | 5 | aa2ar example | 18.4| 0.23 |
| DGXC 8xA100 | 5 | abl1 example |10.5 | 0.26 |

Notes: 
- aa2ar example: 80 compounds, split into 8 batches of 10 compounds/batch
- abl1 example: 40 compounds, split into 8 batches of 5 compounds/batch
- `per ligand runtime = total runtime/number of ligands`. 

